In [ ]:
import syft as sy
import torch
from tools import models
import numpy as np
import pandas as pd
import time

sy.load('opacus')
np.random.seed(42) # The meaning of life!

#### Connect to to Data Owner 1

In [ ]:
duet1 = sy.launch_duet(loopback=True)

#### Connect to to Data Owner 2

In [ ]:
duet2 = sy.launch_duet(loopback=True)

#### Get the Pointers to the data

In [ ]:
# Make list of all duets
duets = [duet1, duet2]

# Get pointers from store
train_ptrs = [(duet.store[0], duet.store[1]) for duet in duets]
test_ptrs = [(duet.store[2], duet.store[3]) for duet in duets]

#### Constants for training and tracking

In [ ]:
#Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = True # Whether or not this run should be tracked in the results csv file
DP = False # Whether or not Differential Privacy should be applied

# Parameters for training and Differential Privacy
BATCH_SIZE = 100
EPOCHS = 10
LEARNING_RATE = 0.002 if DP else 0.001

DELTA = 1e-3 # Set to be less then the inverse of the size of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)
NOISE_MULTIPLIER = 2.0 # The amount of noise sampled and added to the average of the gradients in a batch (from https://opacus.ai/tutorials/building_image_classifier)
MAX_GRAD_NORM = 1.2 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step (from https://opacus.ai/tutorials/building_image_classifier)

length = len(train_data_ptr)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

#### Setup Models and Differential Privacy Engine

In [ ]:
# Setups contains per duet/DO one set of: 
# 1. remote_model  2. remote_torch  3. optimizer  4. loss_function  5. device   6. privacy_engine_ptr  
setups = [] 

for duet in duets:
    # Getting remote and local instances
    local_model = models.Deep2DNet(torch)
    remote_model = local_model.send(duet)
    remote_torch = duet.torch
    remote_opacus = duet.opacus
    
    # Setting device to train on
    cuda_available = remote_torch.cuda.is_available().get(request_block=True, reason='Need to check for available GPU!')
    if cuda_available:
        device = remote_torch.device('cuda:0')
        remote_model.cuda(device)
    else:
        device = remote_torch.device('cpu')
        remote_model.cpu()

    # Optimizer and Loss Function
    params = remote_model.parameters()
    optim = remote_torch.optim.Adam(params=params, lr=LEARNING_RATE)
    loss_function = remote_torch.nn.CrossEntropyLoss()

    # Setting up Differential Privacy Engine
    if DP:
        privacy_engine_ptr = remote_opacus.privacy_engine.PrivacyEngine(
            remote_model.real_module, sample_rate=SAMPLE_RATE,
            noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
        )
        privacy_engine_ptr.to(device)
        privacy_engine_ptr.attach(optim)
    else:
        privacy_engine_ptr = None
        
    setup = [remote_model, remote_torch, optimizer, loss_function, device, privacy_engine_ptr]
    setups.append(setup)


#### Train the Models at each Data Owner

In [ ]:
for train, test, setup in zip(train_ptrs, test_ptrs, setups):
    
    losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, DELTA, 
                                                                          setup[0], setup[1],
                                                                          setup[2], setup[3], 
                                                                          train[0], train[1], 
                                                                          test[0], test[1], 
                                                                          [1, 64, 64], setup[4], setup[5])
    
    
    
    